In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [4]:
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using mps device


# 클래스 정의하기

In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [12]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [14]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1], device='mps:0')


# 모델 계층 (Layer)

In [15]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


## nn.Flatten()

In [16]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear()

In [21]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## nn.ReLU()

In [22]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2088,  0.0294, -0.1441,  0.1399,  0.2695,  0.0350,  0.2805, -0.0767,
         -0.1105, -0.2123, -0.1893, -0.3991,  0.0500, -0.5587,  0.0988,  0.2145,
         -0.1087, -0.3886, -0.1992,  0.5461],
        [-0.0283,  0.3177,  0.1485,  0.2202,  0.2693,  0.0112,  0.1871, -0.1125,
         -0.1716,  0.1197, -0.0430,  0.0354, -0.5281, -0.8030,  0.0162,  0.1827,
         -0.1875, -0.2549, -0.2252,  0.6758],
        [ 0.1303,  0.0956,  0.2360, -0.0101,  0.5512, -0.0811, -0.1824, -0.0437,
         -0.0920, -0.1291, -0.1328, -0.2062, -0.2836, -0.7369, -0.1921,  0.1203,
         -0.0592, -0.2174, -0.1172,  0.2859]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0294, 0.0000, 0.1399, 0.2695, 0.0350, 0.2805, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0500, 0.0000, 0.0988, 0.2145, 0.0000, 0.0000,
         0.0000, 0.5461],
        [0.0000, 0.3177, 0.1485, 0.2202, 0.2693, 0.0112, 0.1871, 0.0000, 0.0000,
         0.1197, 0.0000, 0.0354, 0.0000, 0.0000, 0.01

## nn.Softmax()

In [23]:
softmax = nn.Softmax(dim=1)
pred_softmax = softmax(logits)

## 모델 매개변수

In [24]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0253,  0.0333, -0.0051,  ..., -0.0140, -0.0039,  0.0117],
        [-0.0323,  0.0240, -0.0100,  ...,  0.0330,  0.0103, -0.0333]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0300, 0.0160], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0400,  0.0086, -0.0344,  ..., -0.0204, -0.0284,  0.0365],
        [ 0.0134, -0.0102,  0.0300,  ..., -0.0244, -0.0274, -0.0138]],
       device='mps:0', grad_fn=<SliceBa